In [1]:
a = 2151264638 % 16
print (a)

14


In [2]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, RobustScaler,PowerTransformer
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.naive_bayes import GaussianNB
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score, accuracy_score, classification_report
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import metrics
import lightgbm as lgb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from filterpy.kalman import KalmanFilter #Kalman
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
df=pd.read_csv("Gia_SMP_TS21_5_2024.csv",index_col = 'Ng?y')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x88 in position 2: invalid start byte

In [ ]:
df.head()

In [ ]:
feats= ['15', '16', '17']

In [ ]:
transformer = PowerTransformer()
X=transformer.fit_transform(df[feats])

In [ ]:
pca = PCA(n_components=0.5,random_state=1)
pca.fit(X)
PCA_ds = pd.DataFrame(pca.transform(df[feats]), columns=(["col"]))
PCA_ds.describe().T

In [ ]:
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(random_state=23), k=(4,12))
Elbow_M.fit(X)
Elbow_M.show()

In [ ]:
# USING GAUSIAN
BGM = BayesianGaussianMixture(n_components=7,covariance_type='full',random_state=1,n_init=15)
# fit model and predict clusters
preds = BGM.fit_predict(X)
PCA_ds["Clusters"] = preds
#Adding the Clusters feature to the orignal dataframe.
df["Clusters"]= preds

In [ ]:
pp=BGM.predict_proba(X)# Calcualting the probabilities of each prediction
df_new=pd.DataFrame(X,columns=feats) 
df_new[[f'predict_proba_{i}' for i in range(6)]]=pp # creating new dataframe columns of probabilites 
df_new['preds']=preds
df_new['predict_proba']=np.max(pp,axis=1)
df_new['predict']=np.argmax(pp,axis=1)
    
train_index=np.array([])
for n in range(7):
    n_inx=df_new[(df_new.preds==n) & (df_new.predict_proba > 0.68)].index
    train_index = np.concatenate((train_index, n_inx))

In [ ]:
X_new=df_new.loc[train_index][feats]
y=df_new.loc[train_index]['preds']

params_lgb = {'learning_rate': 0.06,'objective': 'multiclass','boosting': 'gbdt','n_jobs': -1,'verbosity': -1, 'num_classes':7} 

model_list=[]

gkf = StratifiedKFold(11)
for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_new,y)):   

    tr_dataset = lgb.Dataset(X_new.iloc[train_idx],y.iloc[train_idx],feature_name = feats)
    vl_dataset = lgb.Dataset(X_new.iloc[valid_idx],y.iloc[valid_idx],feature_name = feats)
    
    model = lgb.train(params = params_lgb, 
                train_set = tr_dataset, 
                valid_sets =  vl_dataset, 
                num_boost_round = 5000, 
                callbacks=[ lgb.early_stopping(stopping_rounds=300, verbose=False), lgb.log_evaluation(period=200)])  
    
    model_list.append(model) 

In [ ]:
labels=np.argmax(lgb_preds,axis=1)

In [ ]:
df.Predicted=np.argmax(lgb_preds,axis=1)

In [ ]:
# USING EDA
# Display basic information about the dataset
print(df.info())

# Display summary statistics
print(df.describe())

# Display the number of missing values in each column
print(df.isnull().sum())

In [ ]:
# Handling missing values (example: filling with mean or removing)
df.fillna(df.mean(), inplace=True)
# data.dropna(inplace=True)  # Alternatively, remove rows with missing values

# Convert data types if needed
# data['column_name'] = data['column_name'].astype('desired_type')

# Remove duplicates
df.drop_duplicates(inplace=True)


In [ ]:
# Summary statistics for numerical columns
print(df.describe())

# Summary statistics for categorical columns
print(df.describe())

# Display the distribution of a specific column
print(df.value_counts())


In [ ]:
# Histograms for numerical columns
df.hist(figsize=(10, 8))
plt.show()

# Boxplots to visualize outliers
plt.figure(figsize=(10, 6))
sns.boxplot(data=df)
plt.xticks(rotation=90)
plt.show()

# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

# Pairplot to visualize relationships
sns.pairplot(df)
plt.show()

# Specific plots for categorical data
plt.figure(figsize=(10, 6))
sns.countplot(df['categorical_column'])
plt.show()

# Scatter plot for two numerical columns
plt.figure(figsize=(10, 6))
sns.scatterplot(x='9', y='10', data=df)
plt.show()


In [ ]:
# Calculate correlation matrix
corr_matrix = df.corr()

# Display correlation matrix
print(corr_matrix)

# Visualize correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()


In [ ]:
# Using IQR to detect outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Filter out outliers
df_outliers_filtered = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

print(f'Original data shape: {df.shape}')
print(f'Data shape after outlier removal: {df_outliers_filtered.shape}')


In [ ]:
# Using Kamal
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data
np.random.seed(0)
n_timesteps = 50
true_position = np.linspace(0, 100, n_timesteps)
measurements = true_position + np.random.normal(0, 10, n_timesteps)

plt.plot(true_position, label='True Position')
plt.plot(measurements, label='Measurements', linestyle='dashed')
plt.legend()
plt.show()


In [ ]:
import numpy as np

def kalman_filter(measurements, process_variance, measurement_variance, estimated_measurement_variance):
    n_timesteps = len(measurements)
    
    # Allocate space for arrays
    estimated_position = np.zeros(n_timesteps)
    estimated_error = np.zeros(n_timesteps)
    kalman_gain = np.zeros(n_timesteps)
    
    # Initial guesses
    estimated_position[0] = measurements[0]
    estimated_error[0] = 1.0
    
    for t in range(1, n_timesteps):
        # Prediction step
        estimated_position[t] = estimated_position[t-1]
        estimated_error[t] = estimated_error[t-1] + process_variance
        
        # Update step
        kalman_gain[t] = estimated_error[t] / (estimated_error[t] + measurement_variance)
        estimated_position[t] += kalman_gain[t] * (measurements[t] - estimated_position[t])
        estimated_error[t] = (1 - kalman_gain[t]) * estimated_error[t]
    
    return estimated_position, estimated_error, kalman_gain

# Parameters
process_variance = 1.0  # Variance in the process
measurement_variance = 10.0  # Variance in the measurements
estimated_measurement_variance = 1.0  # Initial estimation of the measurement variance

# Apply Kalman filter
estimated_position, estimated_error, kalman_gain = kalman_filter(
    measurements, process_variance, measurement_variance, estimated_measurement_variance
)

# Plot results
plt.plot(true_position, label='True Position')
plt.plot(measurements, label='Measurements', linestyle='dashed')
plt.plot(estimated_position, label='Kalman Filter Estimate')
plt.legend()
plt.show()


In [ ]:
# USING ARIMA

from statsmodels.tsa.arima.model import ARIMA
# Load dataset
data = pd.read_csv('Gia_SMP_TS21_5_2024.csv', index_col='Ng?y')

In [ ]:
train_size = int(len(data) * 0.8)  # 80% for training, 20% for testing
train_data, test_data = data[:train_size], data[train_size:]


In [ ]:
# Define ARIMA model parameters (p, d, q)
p = 5  # Autoregression (AR) order
d = 1  # Integrated (I) order (degree of differencing)
q = 0  # Moving Average (MA) order

# Initialize and fit ARIMA model
arima_model = ARIMA(train_data, order=(p, d, q))
arima_result = arima_model.fit()


In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test_data, predictions))
print(f'Root Mean Squared Error (RMSE): {rmse}')


In [ ]:
# Plot observed vs. predicted values
plt.plot(test_data, label='Observed')
plt.plot(test_data.index, predictions, color='red', label='Predicted')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('ARIMA Model Forecast')
plt.legend()
plt.show()


In [ ]:
pip install statsmodels